In [1]:
from crossword import *
structure = "data/structure0.txt"
words = "data/words0.txt"
crossword = Crossword(structure, words)

In [2]:
class CrosswordCreator():

    def __init__(self, crossword):
        """
        Create new CSP crossword generate.
        """
        self.crossword = crossword
        # self.domains looks like dict with keys = Variable-object and values = copy-set of all words
        self.domains = {
            var: self.crossword.words.copy()
            for var in self.crossword.variables
        }
    
    # Used for print() and save()
    def letter_grid(self, assignment):
        """
        Return 2D array representing a given assignment.
        """
        letters = [
            [None for _ in range(self.crossword.width)]
            for _ in range(self.crossword.height)
        ]
        for variable, word in assignment.items():
            direction = variable.direction
            for k in range(len(word)):
                i = variable.i + (k if direction == Variable.DOWN else 0)
                j = variable.j + (k if direction == Variable.ACROSS else 0)
                letters[i][j] = word[k]
        return letters
    
    def print(self, assignment):
        """
        Print crossword assignment to the terminal.
        """
        letters = self.letter_grid(assignment)
        for i in range(self.crossword.height):
            for j in range(self.crossword.width):
                if self.crossword.structure[i][j]:
                    print(letters[i][j] or " ", end="")
                else:
                    print("█", end="")
            print()

    def save(self, assignment, filename):
        """
        Save crossword assignment to an image file.
        """
        from PIL import Image, ImageDraw, ImageFont
        cell_size = 100
        cell_border = 2
        interior_size = cell_size - 2 * cell_border
        letters = self.letter_grid(assignment)

        # Create a blank canvas
        img = Image.new(
            "RGBA",
            (self.crossword.width * cell_size,
             self.crossword.height * cell_size),
            "black"
        )
        font = ImageFont.truetype("assets/fonts/OpenSans-Regular.ttf", 80)
        draw = ImageDraw.Draw(img)

        for i in range(self.crossword.height):
            for j in range(self.crossword.width):

                rect = [
                    (j * cell_size + cell_border,
                     i * cell_size + cell_border),
                    ((j + 1) * cell_size - cell_border,
                     (i + 1) * cell_size - cell_border)
                ]
                if self.crossword.structure[i][j]:
                    draw.rectangle(rect, fill="white")
                    if letters[i][j]:
                        w, h = draw.textsize(letters[i][j], font=font)
                        draw.text(
                            (rect[0][0] + ((interior_size - w) / 2),
                             rect[0][1] + ((interior_size - h) / 2) - 10),
                            letters[i][j], fill="black", font=font
                        )

        img.save(filename)

In [3]:
creator = CrosswordCreator(crossword)

In [4]:
def revise(x, y):
    """
    Make variable `x` arc consistent with variable `y`.
    To do so, remove values from `self.domains[x]` for which there is no
    possible corresponding value for `y` in `self.domains[y]`.

    Return True if a revision was made to the domain of `x`; return
    False if no revision was made.
    """
    """
    function revise(csp, x, y):
        revised = False
        for xi in x.domain:
            if no yi in y.domain satisfies constraint for (x,y):
                delete xi from x.domain
                revised = True
        return revised
    """
    """
    for xi: for yi to sastify constraint = at least 1 yi != xi
    if all yi == xi than constraint not satisfied
    if all yi == xi than delete xi
    code:
    for yi in y.domain:
        while True:
            if yi == xi:
                continue
            else:
                x.domain.remove(xi)
                break

    """

    revised = False

    # List with removing candidates
    removing_list = []
    
    # Indices of overlapping
    i_x, i_y = crossword.overlaps[x, y]
    
    for word_x in creator.domains[x]:
        print("starting here X: ", word_x, ", overlap:", word_x[i_x])

        # Count if condition for arc consistency was met: word_y != word_x
        count = 0

        for word_y in creator.domains[y]:
            print("word_y = ", word_y, ", overlap:", word_y[i_y])
            if word_y[i_y] == word_x[i_x]:
                print("overlapping: True")
                count += 1
                break

        # If condition was not met add word_x to removing list
        if count == 0:
            print("removing: ", word_x)
            # creator.domains[x].remove(word_x)
            removing_list.append(word_x)
            revised = True

    # Remove candidated from set domain[x]
    creator.domains[x] = creator.domains[x] - set(removing_list)

    return revised

In [5]:
VarX = Variable(0, 1, 'across', 3)
VarY = Variable(0, 1, 'down', 5)

In [6]:
creator.domains[VarY].remove('ONE')

In [7]:

revise(x = VarX, y = VarY)

starting here X:  EIGHT , overlap: E
word_y =  EIGHT , overlap: E
overlapping: True
starting here X:  ONE , overlap: O
word_y =  EIGHT , overlap: E
word_y =  FOUR , overlap: F
word_y =  SIX , overlap: S
word_y =  THREE , overlap: T
word_y =  TEN , overlap: T
word_y =  FIVE , overlap: F
word_y =  TWO , overlap: T
word_y =  SEVEN , overlap: S
word_y =  NINE , overlap: N
removing:  ONE
starting here X:  FOUR , overlap: F
word_y =  EIGHT , overlap: E
word_y =  FOUR , overlap: F
overlapping: True
starting here X:  SIX , overlap: S
word_y =  EIGHT , overlap: E
word_y =  FOUR , overlap: F
word_y =  SIX , overlap: S
overlapping: True
starting here X:  THREE , overlap: T
word_y =  EIGHT , overlap: E
word_y =  FOUR , overlap: F
word_y =  SIX , overlap: S
word_y =  THREE , overlap: T
overlapping: True
starting here X:  TEN , overlap: T
word_y =  EIGHT , overlap: E
word_y =  FOUR , overlap: F
word_y =  SIX , overlap: S
word_y =  THREE , overlap: T
overlapping: True
starting here X:  FIVE , overlap

True

In [8]:
creator.domains[VarX]

{'EIGHT', 'FIVE', 'FOUR', 'NINE', 'SEVEN', 'SIX', 'TEN', 'THREE', 'TWO'}

Test implemented revise()

In [2]:
from generate import *

In [3]:
creator = CrosswordCreator(crossword)

In [5]:
creator.revise(x = VarX, y = VarY)

False